# MDAnalysis Tutorial 

## Preparations

### Loading MDAnalysis

In [1]:
#!which python
import numpy as np
import MDAnalysis as mda
#from MDAnalysis.tests.datafiles import PSF, DCD, GRO, TRR
from MDAnalysis.analysis import rms
import pandas as pd
#import nglview as nv
import MDAnalysis.transformations
from MDAnalysis.analysis import diffusionmap, align, rms
import os
%matplotlib inline
import matplotlib.pyplot as plt

In [2]:
mda.__version__

'2.6.1'

In [3]:
!ls

MDAnalysisTrajectoryGlobalAlphaSyn.ipynb
PlottingRgRMSD.ipynb
trajMolearn_stride_proteinH_center_unwrap.pdb


## Basics

### Universe, AtomGroup and Trajectory

MDAnalysis is an object oriented library, meaning that data from your molecular simulations are made available through various MDAnalysis objects, which will be introduced here.

The first object in any analysis is the `Universe` object, which is the central data structure from which all other objects are created.  This is created through providing a topology (description of the system) and trajectory (description of the temporal data) file.

In [4]:
u = mda.Universe('../trajMolearn_stride_proteinH_center_unwrap.pdb', dt=1000)
#protein = u.select_atoms("protein and not name H*")
print(u)

<Universe with 439 atoms>


In [5]:
aligner = align.AlignTraj(u, u, select='protein and not name H*',
                          in_memory=True).run()

In [6]:
ag = u.select_atoms('all')
with MDAnalysis.Writer("alignToProteinH.pdb", ag.n_atoms) as W:
    for ts in u.trajectory:
        W.write(ag)

/home/marmatt/miniforge3/envs/molearn/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1151: UserWarning: Found no information for attr: 'formalcharges' Using default value of '0'
  warnings.warn("Found no information for attr: '{}'"
/home/marmatt/miniforge3/envs/molearn/lib/python3.10/site-packages/MDAnalysis/coordinates/PDB.py:1198: UserWarning: Found missing chainIDs. Corresponding atoms will use value of 'X'
  warnings.warn("Found missing chainIDs."


In [7]:
# Calculate radius of gyration in all frames of trajectory

Rgyr2KKW = []

for ts in u.trajectory:
    Rgyr2KKW.append((u.trajectory.time, u.atoms.radius_of_gyration()))
    
Rgyr2KKW = np.array(Rgyr2KKW)
Rgyr2KKW[:,0] = Rgyr2KKW[:,0]/1000 #make sure units are in ns, must be run once only
df_alpha_syn_rg = pd.DataFrame(Rgyr2KKW, columns=['Time (ns)', 'Rg'])
df_alpha_syn_rg.to_csv('alpha-syn_Rg_data.csv')

In [8]:
from MDAnalysis.analysis import diffusionmap, align, rms
#calculate pairwise-RMSD
matrix = diffusionmap.DistanceMatrix(u, select='name CA').run()

df_alpha_syn_pairRMSD = pd.DataFrame(matrix.dist_matrix)

df_alpha_syn_pairRMSD.to_csv('alpha-syn_pairRMSD_data.csv')

/home/marmatt/miniforge3/envs/molearn/lib/python3.10/site-packages/MDAnalysis/analysis/diffusionmap.py:288: DeprecationWarning: The `dist_matrix` attribute was deprecated in MDAnalysis 2.0.0 and will be removed in MDAnalysis 3.0.0. Please use `results.dist_matrix` instead.
  warnings.warn(wmsg, DeprecationWarning)
